## Importsetup

In [17]:


###########################################################################
from app.main.extraction.dxf_extraction_coordinator import DxfExtractionCoordinator
from app.main.models.wall import Wall 
from app.main.models.architecture_wall import Architectur_Wall
from app.main.models.point import Point 
from app.main.models.window import Window
from app.main.services.walls_extraction import WallsExtraction
from app.main.services.dxf_extraction_simple import DxfExtractionSimple
from app.main.dtos.request.extraction_request_file import ExtractionRequestFile
from app.main.models.line import Line 
from app.main.constants import (PLOT_DPI, PLOT_LINE_WIDTH_LARGE,
                                PLOT_LINE_WIDTH_SMALL, DefaultColors,
                                OrientationColors, PlotStyle, PlotTypes)
###########################################################################
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon as shapePolygon
from shapely.geometry import Point as shapePoint
from shapely.geometry import LineString
import math
import numpy as np
import itertools as it
from scipy.spatial.distance import cdist
import ezdxf
import uuid

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import operator

###########################################################################
file_path_simple = '../../media/Einfache_Plaene_Compass/'
file_path_architecture = '../../media/Architektenplaene/'
###########################################################################
file_name_simple = 'R-Bau_D_EG.dxf'
file_name_architecutre = 'R-Bau_A_2OG.dxf'
###########################################################################
simple_dxf = ezdxf.readfile(file_path_simple+file_name_simple)
architecture_dxf = ezdxf.readfile(file_path_architecture+file_name_architecutre)
simple_model = simple_dxf.modelspace()
arichtecture_model = architecture_dxf.modelspace()

## Creating dxf files for analysis

In [18]:
#Read pillar circles from architecture wall
architecture_polylines = []
architecture_lines = []
fassaden_polylines = arichtecture_model.query("LWPolyline[layer=='A_05_FASSADE_ELEMENTE']")#.first.get_points('xy')

#lines
fassaden_lines = arichtecture_model.query("LINE[layer=='A_05_FASSADE_ELEMENTE']")

#Arcs
fassaden_arc = arichtecture_model.query("ARC[layer=='A_05_FASSADE_ELEMENTE']")

for polyline in fassaden_polylines:
    architecture_polylines.append(polyline)
for line in fassaden_lines:
    architecture_lines.append(line)

doc = ezdxf.new('R2000')
msp = doc.modelspace()
doc.layers.new(name='FASSADE_LINES', dxfattribs={'linetype': 'DASHED', 'color': 1})
doc.layers.new(name='FASSADE_POLYLINES', dxfattribs={'linetype': 'DASHED', 'color': 2})
doc.layers.new(name='FASSADE_ARCS', dxfattribs={'linetype': 'DASHED', 'color': 3})

#linetypes = ['BYLAYER', 'HIDDEN', '4.102_EG$0$CONTINOUS']
linetypes = ['HIDDEN']
for x in architecture_polylines:
    #mylinetype = x.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if x.dxf.linetype in linetypes:
        msp.add_lwpolyline(x, dxfattribs={'layer': 'FASSADE_POLYLINES'})

#add lines to drawing
coordinates = []

for e in architecture_lines:
    #mylinetype = e.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if e.dxftype() == 'LINE' and e.dxf.linetype in linetypes:
        tupel = (e.dxf.start, e.dxf.end)
        coordinates.append(tupel)

for x in coordinates:
    msp.add_line(x[0], x[1], dxfattribs={'layer': 'FASSADE_LINES'})


#add arcs to drwawing
for x in fassaden_arc:
    #mylinetype = x.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if  x.dxf.linetype in linetypes:
        msp.add_arc(x.dxf.center, x.dxf.radius, x.dxf.start_angle, x.dxf.end_angle, True, dxfattribs={'layer': 'FASSADE_ARCS'})

print(linetypes)
doc.saveas("facade_dashed_2og.dxf")


['HIDDEN']


In [3]:
'''This method gets the length between points'''
def get_length_between_points(point_1: Point, point_2: Point) -> float:
    line_string = LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])
    return line_string.length

In [4]:
'''This method creates a point from a coordinate'''
def create_point_from_coordinate(coordinate: tuple) -> Point:
    return Point(x_coordinate=coordinate[0], y_coordinate=coordinate[1])

In [5]:
'''This method creates a window from an ezdxf arc'''
def create_window_from_arc(arc) -> Window:
    window_id = str(uuid.uuid4())
    angle = abs(arc.dxf.start_angle - arc.dxf.end_angle)
    center_point = create_point_from_coordinate(arc.dxf.center)
    start_point = create_point_from_coordinate(arc.start_point)
    end_point = create_point_from_coordinate(arc.end_point)
    radius = arc.dxf.radius
    return Window(window_id, center_point, start_point, end_point, radius, angle)

In [6]:
# testing the method
windows = []
for arc in fassaden_arc:
    if arc.dxf.radius <=2.0:
        window = create_window_from_arc(arc)
        windows.append(window)


In [7]:
'''This method creates a line string from 2 points'''
def create_line_string_of_two_points(point_1: Point, point_2: Point)-> LineString:
    return LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])

## Matching windows

In [15]:
#loop through windows to match them
dxf_file = ezdxf.readfile(file_path_simple + file_name_simple)
dxf_extraction_simple = DxfExtractionSimple()
simple_polygons, polygons_with_virtual_entities, extracted_functionalities = dxf_extraction_simple.get_all_polygons(dxf_file ,file_path_simple + file_name_simple)

for window in windows:
    window.amount_matched = 0
    window.is_matched = False
    #create 2 linestrings
    line_string_start = create_line_string_of_two_points(window.center_point, window.start_point)
    line_string_end = create_line_string_of_two_points(window.center_point, window.end_point)
    #loop through polygons
    for polygon in simple_polygons:
        matched = False
        for wall in polygon.walls:
            line_string_wall =create_line_string_of_two_points(wall.start_point, wall.end_point)
            if line_string_wall.intersects(line_string_start) or line_string_wall.intersects(line_string_end):
                window.is_matched = True
                matched = True
        if matched:
            if not polygon.windows:
                polygon.windows = []
            window.amount_matched = window.amount_matched + 1
            polygon.windows.append(window)


#second run
for polygon in simple_polygons:
    for wall in polygon.walls:
        line_string_wall =create_line_string_of_two_points(wall.start_point, wall.end_point)
        for window in windows:
            if window.is_matched:
                continue 
            #create 2 linestrings
            line_string_start = create_line_string_of_two_points(window.center_point, window.start_point)
            line_string_end = create_line_string_of_two_points(window.center_point, window.end_point)
            distance_center_start_wall = line_string_start.distance(line_string_wall)
            distance_center_end_wall = line_string_end.distance(line_string_wall)
            
            if distance_center_start_wall <= 0.5 or distance_center_end_wall <= 0.5:
                window.is_matched = True
                if not polygon.windows:
                    polygon.windows = []
                    polygon.windows.append(window)
                else:
                    window_already_in_polygon = False
                    for poly_window in polygon.windows:
                        if window.window_id == poly_window.window_id:
                            window_already_in_polygon = True
                    if not window_already_in_polygon:
                        polygon.windows.append(window)

counter = 0
for window in windows:
    if not window.is_matched:
        counter = counter + 1
    
print("Anzahl ungematchter Fenster: {}".format(str(counter)))
print("Anzahl Fenster insgesamt: {}".format(str(len(windows))))
            


    

2021-06-06 19:43:18.293 | SUCCESS  | app.main.services.orientation_extraction:get_north_orientation:117 - Compass extraction successful, return coordinate system
2021-06-06 19:43:18.294 | INFO     | app.main.services.dxf_extraction_simple:get_all_polygons:78 - Orientation will be calculated.
2021-06-06 19:43:18.294 | SUCCESS  | app.main.services.extraction_helper:get_modelspace:190 - sucessfully get_modelspace() from dxf_file: <ezdxf.document.Drawing object at 0x13c6d84c0>
2021-06-06 19:43:18.295 | DEBUG    | app.main.services.extraction_helper:get_outline_polygon:199 - get_outline_polygon for modelspace: <ezdxf.layouts.layout.Modelspace object at 0x14140a220>
2021-06-06 19:43:18.318 | DEBUG    | app.main.services.dxf_extraction_simple:get_all_polygons:91 - got all room_polylines on layer: RAUMPOLYGON
Anzahl ungematchter Fenster: 7
Anzahl Fenster insgesamt: 227


In [23]:
'''This method plots unmatched windows'''
def plot_unmatched_windows(windows: list[Window], polygons: list):
    
    for window in windows:
        if not window.is_matched:
            plt.plot([window.start_point.x_coordinate, window.end_point.x_coordinate], [window.start_point.y_coordinate, window.end_point.y_coordinate], color=DefaultColors.GRAY.value, linewidth=PLOT_LINE_WIDTH_LARGE)
            plt.plot([window.center_point.x_coordinate, window.end_point.x_coordinate], [window.center_point.y_coordinate, window.end_point.y_coordinate], color=DefaultColors.RED.value, linewidth=PLOT_LINE_WIDTH_LARGE)
            plt.plot([window.center_point.x_coordinate, window.start_point.x_coordinate], [window.center_point.y_coordinate, window.start_point.y_coordinate], color=DefaultColors.GREEN.value, linewidth=PLOT_LINE_WIDTH_LARGE)
    
    for polygon in polygons:
        for wall in polygon.walls:
            plt.plot([wall.start_point.x_coordinate,wall.end_point.x_coordinate], [wall.start_point.y_coordinate, wall.end_point.y_coordinate], color="k", linewidth=PLOT_LINE_WIDTH_LARGE)
    plt.tick_params(axis=PlotStyle.AXIS.value, left=PlotStyle.TICKS_LEFT.value, bottom=PlotStyle.TICKS_BOTTOM.value, labelleft=PlotStyle.LABEL_LEFT.value, labelbottom=PlotStyle.LABEL_BOTTOM.value)
    plt.savefig("windows_unmatched", dpi=PLOT_DPI)
    plt.clf()

'''This method plots duplicate windows'''
def plot_duplicate_windows(windows: list[Window], polygons: list):
    for window in windows:
        plt.plot([window.start_point.x_coordinate, window.end_point.x_coordinate], [window.start_point.y_coordinate, window.end_point.y_coordinate], color=DefaultColors.GRAY.value, linewidth=PLOT_LINE_WIDTH_LARGE)
        plt.plot([window.center_point.x_coordinate, window.end_point.x_coordinate], [window.center_point.y_coordinate, window.end_point.y_coordinate], color=DefaultColors.RED.value, linewidth=PLOT_LINE_WIDTH_LARGE)
        plt.plot([window.center_point.x_coordinate, window.start_point.x_coordinate], [window.center_point.y_coordinate, window.start_point.y_coordinate], color=DefaultColors.GREEN.value, linewidth=PLOT_LINE_WIDTH_LARGE)
    
    for polygon in polygons:
        for wall in polygon.walls:
            plt.plot([wall.start_point.x_coordinate,wall.end_point.x_coordinate], [wall.start_point.y_coordinate, wall.end_point.y_coordinate], color="k", linewidth=PLOT_LINE_WIDTH_LARGE)
    plt.tick_params(axis=PlotStyle.AXIS.value, left=PlotStyle.TICKS_LEFT.value, bottom=PlotStyle.TICKS_BOTTOM.value, labelleft=PlotStyle.LABEL_LEFT.value, labelbottom=PlotStyle.LABEL_BOTTOM.value)
    plt.savefig("windows_duplicate", dpi=PLOT_DPI)
    plt.clf()
'''This method plots a list of windows'''
def plot_windows(windows: list[Window]):
    
    for window in windows:
        plt.plot([window.start_point.x_coordinate, window.end_point.x_coordinate], [window.start_point.y_coordinate, window.end_point.y_coordinate], color=DefaultColors.GRAY.value, linewidth=PLOT_LINE_WIDTH_LARGE)
        plt.plot([window.center_point.x_coordinate, window.end_point.x_coordinate], [window.center_point.y_coordinate, window.end_point.y_coordinate], color=DefaultColors.RED.value, linewidth=PLOT_LINE_WIDTH_LARGE)
        plt.plot([window.center_point.x_coordinate, window.start_point.x_coordinate], [window.center_point.y_coordinate, window.start_point.y_coordinate], color=DefaultColors.GREEN.value, linewidth=PLOT_LINE_WIDTH_LARGE)
    plt.tick_params(axis=PlotStyle.AXIS.value, left=PlotStyle.TICKS_LEFT.value, bottom=PlotStyle.TICKS_BOTTOM.value, labelleft=PlotStyle.LABEL_LEFT.value, labelbottom=PlotStyle.LABEL_BOTTOM.value)
    plt.savefig("windows", dpi=PLOT_DPI)
    plt.clf()





## Plotting results

In [25]:
plot_windows(windows)
#plot_unmatched_windows(windows, simple_polygons)
special_windows = []

for window in windows:
    if window.min_wall_distance:
        if window.min_wall_distance > 0.5:
            special_windows.append(window)
plot_unmatched_windows(special_windows, simple_polygons)
counter = 0
for polygon in simple_polygons:
    if polygon.windows:
        counter = counter + len(polygon.windows)
print("Anzahl Fenster {}".format(str(counter)))
flat_windows = []
for polygon in simple_polygons:
    if polygon.windows:
        for window in polygon.windows:
            flat_windows.append(window.window_id)
duplicate_windows = set([x for x in flat_windows if flat_windows.count(x) > 1])
print(duplicate_windows)

#duplicates = {'910de528-bef7-4214-a717-7cb86cbd3580', 'ee5a0c39-3ae0-48e1-99da-a87077c11514'}
duplicate_window_objects = []
for polygon in simple_polygons:
    if polygon.windows:
        for window in polygon.windows:
            if window.window_id == '910de528-bef7-4214-a717-7cb86cbd3580' or window.window_id =='ee5a0c39-3ae0-48e1-99da-a87077c11514':
                duplicate_window_objects.append(window)
plot_duplicate_windows(duplicate_window_objects, simple_polygons)
print(duplicate_window_objects)

Anzahl Fenster 222
{'910de528-bef7-4214-a717-7cb86cbd3580', 'ee5a0c39-3ae0-48e1-99da-a87077c11514'}
[Window(window_id='910de528-bef7-4214-a717-7cb86cbd3580', center_point=Point(x_coordinate=-24.65481884964987, y_coordinate=24.41316514892236, s_width=None, e_width=None, b_value=None), start_point=Point(x_coordinate=-24.65481884964987, y_coordinate=25.55316514892236, s_width=None, e_width=None, b_value=None), end_point=Point(x_coordinate=-25.79481884964987, y_coordinate=24.41316514892236, s_width=None, e_width=None, b_value=None), radius=1.14, angle=90.0, is_matched=True, amount_matched=2, min_wall_distance=None), Window(window_id='ee5a0c39-3ae0-48e1-99da-a87077c11514', center_point=Point(x_coordinate=-23.35481884964994, y_coordinate=24.41316514892236, s_width=None, e_width=None, b_value=None), start_point=Point(x_coordinate=-23.35481884964994, y_coordinate=25.55316514892236, s_width=None, e_width=None, b_value=None), end_point=Point(x_coordinate=-24.494818849649942, y_coordinate=24.4131